<a href="https://colab.research.google.com/github/radonys/airbnb/blob/master/AirbnbLA_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Mining and Predictive Analytics



#### Project Title: **Airbnb LA Business Analysis**
---
#### Market Assigned: **Los Angeles**
---

"We, the undersigned, certify that the report submitted is our original work; all authors participated in the work in a substantive way; all authors have seen and approved the report as submitted; the text, images, illustrations, and other items included in the manuscript do not carry any infringement/plagiarism issue upon any existing copyrighted materials."


---

Member ID | Member Name 
--- | ---
Contact Member | Yash Srivastava
Team Member 2 | Shohini Ghosh
Team Member 3 | Manas Mishra
Team Member 4 | Madhura Dighe
Team Member 5 | Pratik Pandey
Team Member 6 | Surbhi Gupta

#### Executive Summary

#### Research Questions

#### Methodology

#### Results and Findings

#### Conclusion and Discussions

#### References

#### Appendix

### Code


#### Libraries

##### Install Libraries

In [0]:
install.packages(c("tidymodels", "plotly", "lubridate", "car", "fpp3", "skimr", "caret"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



##### Load Libraries

In [1]:
library("tidyverse")
library("tidymodels")
library("plotly")
library("skimr")
#library("caret")
library("lubridate")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 0.1.0 ──

✔ broom     0.5.5      ✔ rsample   0.0.6 
✔ dials     0.0.6      ✔ tune      0.1.0 
✔ infer     0.5.1      ✔ workflows 0.1.1 
✔ parsnip   0.1.0      ✔ yardstick 0.0.6 
✔ recipes   0.1.12     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ dials::margin()   masks ggplot2::margin()
✖ yardstick::spec() masks readr::spec()
✖ recip

#### Read Data

In [87]:
train <- read_csv("airbnbTrain.csv")
test <- read_csv("airbnbTest.csv")

Parsed with column specification:
cols(
  .default = col_character(),
  id = col_double(),
  high_booking_rate = col_double(),
  accommodates = col_double(),
  availability_30 = col_double(),
  availability_365 = col_double(),
  availability_60 = col_double(),
  availability_90 = col_double(),
  bathrooms = col_double(),
  bedrooms = col_double(),
  beds = col_double(),
  guests_included = col_double(),
  host_has_profile_pic = col_logical(),
  host_identity_verified = col_logical(),
  host_is_superhost = col_logical(),
  host_listings_count = col_double(),
  host_since = col_date(format = ""),
  instant_bookable = col_logical(),
  is_business_travel_ready = col_logical(),
  is_location_exact = col_logical(),
  latitude = col_double()
  # ... with 15 more columns
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  .default = col_character(),
  id = col_double(),
  accommodates = col_double(),
  availability_30 = col_double(),
  availability_365 =

#### Filter LA Data

In [0]:
trainLA <- train %>% 
    rename(randomControl = "{randomControl}") %>% 
    mutate(randomControl = as.numeric(randomControl)) %>% 
    filter(randomControl > 111999 & randomControl < 113000)

testLA <- test %>% 
    rename(randomControl = "{randomControl}") %>% 
    mutate(randomControl = as.numeric(randomControl)) %>% 
    filter(randomControl > 111999 & randomControl < 113000)

#### Clean Data

##### View Initial Table Properties

In [89]:
skim(trainLA)

── Data Summary ────────────────────────
                           Values 
Name                       trainLA
Number of rows             24075  
Number of columns          66     
_______________________           
Column type frequency:            
  character                31     
  Date                     1      
  logical                  9      
  numeric                  25     
________________________          
Group variables            None   

── Variable type: character ────────────────────────────────────────────────────
   skim_variable         n_missing complete_rate   min   max empty n_unique
 1 access                    10120        0.580      1  1000     0    12325
 2 amenities                     0        1          2  1918     0    22663
 3 bed_type                      0        1          5    13     0        5
 4 cancellation_policy           0        1          6    27     0        9
 5 city                         17        0.999      2    46     0      277
 

##### Character to Numeric Variables

In [90]:
trainLA$price <- as.numeric(gsub('\\$|,', '', trainLA$price))
testLA$price <- as.numeric(gsub('\\$|,', '', testLA$price))

trainLA$security_deposit <- as.numeric(gsub('\\$|,', '', trainLA$security_deposit))
testLA$security_deposit <- as.numeric(gsub('\\$|,', '', testLA$security_deposit))

trainLA$cleaning_fee <- as.numeric(gsub('\\$|,', '', trainLA$cleaning_fee))
testLA$cleaning_fee <- as.numeric(gsub('\\$|,', '', testLA$cleaning_fee))

trainLA$extra_people <- as.numeric(gsub('\\$|,', '', trainLA$extra_people))
testLA$extra_people <- as.numeric(gsub('\\$|,', '', testLA$extra_people))

trainLA$host_response_rate <- as.numeric(gsub('%', '', trainLA$host_response_rate))
testLA$host_response_rate <- as.numeric(gsub('%', '', testLA$host_response_rate))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


##### Handle Missing Values

###### Fill with zeroes

In [0]:
trainLA$host_response_rate[is.na(trainLA$host_response_rate)] <- 0
testLA$host_response_rate[is.na(testLA$host_response_rate)] <- 0

trainLA$host_listings_count[is.na(trainLA$host_listings_count)] <- 0
testLA$host_listings_count[is.na(testLA$host_listings_count)] <- 0

trainLA$review_scores_accuracy[is.na(trainLA$review_scores_accuracy)] <- 0
testLA$review_scores_accuracy[is.na(testLA$review_scores_accuracy)] <- 0

trainLA$review_scores_checkin[is.na(trainLA$review_scores_checkin)] <- 0
testLA$review_scores_checkin[is.na(testLA$review_scores_checkin)] <- 0

trainLA$review_scores_cleanliness[is.na(trainLA$review_scores_cleanliness)] <- 0
testLA$review_scores_cleanliness[is.na(testLA$review_scores_cleanliness)] <- 0

trainLA$review_scores_communication[is.na(trainLA$review_scores_communication)] <- 0
testLA$review_scores_communication[is.na(testLA$review_scores_communication)] <- 0

trainLA$review_scores_location[is.na(trainLA$review_scores_location)] <- 0
testLA$review_scores_location[is.na(testLA$review_scores_location)] <- 0

trainLA$review_scores_value[is.na(trainLA$review_scores_value)] <- 0
testLA$review_scores_value[is.na(testLA$review_scores_value)] <- 0

trainLA$review_scores_rating[is.na(trainLA$review_scores_rating)] <- 0
testLA$review_scores_rating[is.na(testLA$review_scores_rating)] <- 0

###### Fill with False

In [0]:
trainLA$host_identity_verified[is.na(trainLA$host_identity_verified)] <- FALSE
testLA$host_identity_verified[is.na(testLA$host_identity_verified)] <- FALSE

trainLA$host_is_superhost[is.na(trainLA$host_is_superhost)] <- FALSE
testLA$host_is_superhost[is.na(testLA$host_is_superhost)] <- FALSE

###### Fill with Median Date

In [0]:
trainLA$host_since[is.na(trainLA$host_since)] <- as.Date("2015-11-19")
testLA$host_since[is.na(testLA$host_since)] <- as.Date("2015-11-19")

###### Fill with Mean Value

In [0]:
trainLA$cleaning_fee[is.na(trainLA$cleaning_fee)] <- 94.5
testLA$cleaning_fee[is.na(testLA$cleaning_fee)] <- 94.5

trainLA$security_deposit[is.na(trainLA$security_deposit)] <- 490
testLA$security_deposit[is.na(testLA$security_deposit)] <- 490

###### Fill with Median Value

In [0]:
trainLA$bedrooms[is.na(trainLA$bedrooms)] <- 1
testLA$bedrooms[is.na(testLA$bedrooms)] <- 1

trainLA$beds[is.na(trainLA$beds)] <- 1
testLA$beds[is.na(testLA$beds)] <- 1

trainLA$bathrooms[is.na(trainLA$bathrooms)] <- 1
testLA$bathrooms[is.na(testLA$bathrooms)] <- 1

##### Mutate Columns

In [0]:
trainLA <- trainLA %>% mutate(host_days_active = Sys.Date() - host_since)
testLA <- testLA %>% mutate(host_days_active = Sys.Date() - host_since)

trainLA$host_days_active <- as.numeric(trainLA$host_days_active)
testLA$host_days_active <- as.numeric(testLA$host_days_active)

##### Drop Missing-Data/Single-Valued Columns

In [0]:
drop <- c("monthly_price", "square_feet", "randomControl", "weekly_price", "host_acceptance_rate", "state", "is_business_travel_ready", "host_has_profile_pic")

trainLA <- trainLA[ , !(names(trainLA) %in% drop)]
testLA <- testLA[ , !(names(testLA) %in% drop)]

##### Character to Factor

In [0]:
trainLA$bed_type <- as.factor(trainLA$bed_type)
testLA$bed_type <- as.factor(testLA$bed_type)

trainLA$room_type <- as.factor(trainLA$room_type)
testLA$room_type <- as.factor(testLA$room_type)

trainLA$property_type <- as.factor(trainLA$property_type)
testLA$property_type <- as.factor(testLA$property_type)

trainLA$host_response_time <- as.factor(trainLA$host_response_time)
testLA$host_response_time <- as.factor(testLA$host_response_time)

trainLA$cancellation_policy <- as.factor(trainLA$cancellation_policy)
testLA$cancellation_policy <- as.factor(testLA$cancellation_policy)

##### View Final Table Properties

In [99]:
skim(trainLA)

── Data Summary ────────────────────────
                           Values 
Name                       trainLA
Number of rows             24075  
Number of columns          59     
_______________________           
Column type frequency:            
  character                17     
  Date                     1      
  factor                   5      
  logical                  7      
  numeric                  29     
________________________          
Group variables            None   

── Variable type: character ────────────────────────────────────────────────────
   skim_variable         n_missing complete_rate   min   max empty n_unique
 1 access                    10120         0.580     1  1000     0    12325
 2 amenities                     0         1         2  1918     0    22663
 3 city                         17         0.999     2    46     0      277
 4 description                 829         0.966     1  1000     0    22353
 5 host_about                 8937        

In [0]:
write_csv(trainLA, "airbnbTrainLA.csv")
write_csv(testLA, "airbnbTestLA.csv")